In [1]:
mainpingspq = sqlContext.read.load("s3://telemetry-parquet/main_summary/v3", "parquet")

In [13]:
START_profile =17027 #'2016-08-14'
END_profile = 17041#'2016-08-28'
START_pint = '2016-08-14'#do I need this? Sometimes we have subsession_start_date<profile_creation_date, which is weird
END_ping = '2016-10-15'

In [14]:
u1 = mainpingspq.filter(mainpingspq.app_name == 'Firefox').filter(mainpingspq.profile_creation_date >= START_profile).filter(mainpingspq.profile_creation_date <= END_profile).filter(mainpingspq.subsession_start_date.substr(1,10) <= END_ping)

In [15]:
u2 = u1.select(u1.client_id, u1.profile_creation_date, u1.subsession_start_date.substr(1,10).alias('subsession_start_date'), u1.e10s_enabled, u1.e10s_cohort)

In [7]:
u2.count() #823795572

823795572

In [10]:
u3 = u2.filter(u2.e10s_enabled == False)

In [11]:
u3.count() #642566113

642566113

In [11]:
u4 = u2.filter(u2.e10s_enabled == True)

In [14]:
u4.count() #181229457

181229457

In [7]:
sqlContext.registerDataFrameAsTable(u3, "U3")
sqlContext.registerDataFrameAsTable(u4, "U4")

In [8]:
t = sqlContext.sql("""
select subsession_start_date, count(distinct client_id) as num_users
from U3
where profile_creation_date = 17027
group by subsession_start_date
""")

In [10]:
t = t.collect() #takes a really long time

KeyboardInterrupt: 

In [16]:
u2.take(1)

[Row(client_id=u'1a3eaaa4-b5fc-437e-b073-0a61473dd2f6', profile_creation_date=17036, subsession_start_date=u'2016-06-30', e10s_enabled=False, e10s_cohort=u'disqualified-test')]

In [17]:
u3.write.save("bug1314436_e10sDisabled", format="parquet")

KeyboardInterrupt: 

In [18]:
u3.write.save("s3://mozilla-metrics/tmp/bug1314436_e10sDisabled", format="parquet")

KeyboardInterrupt: 

In [ ]:
u4.write.save("s3://mozilla-metrics/tmp/bug1314436_e10sEnabled", format="parquet")

In [37]:
sqlContext.registerDataFrameAsTable(u2,"U")

In [38]:
dau = sqlContext.sql("""
select *
from U
where subsession_start_date >= '{}' and subsession_start_date <= '{}'
and e10s_enabled is not null
""".format(START,END))

In [34]:
dau = dau.collect()

Py4JJavaError: An error occurred while calling o831.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 70 in stage 12.0 failed 4 times, most recent failure: Lost task 70.3 in stage 12.0 (TID 60180, ip-172-31-2-245.us-west-2.compute.internal): org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 1592133. To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:299)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:240)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1145)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:615)
	at java.lang.Thread.run(Thread.java:745)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1431)
	at org.apa

In [ ]:
with open('/tmp/hllMS_dau.json', 'w') as outfile:
    json.dump(dau, outfile)

In [ ]:
call(["aws","s3", "cp", "/tmp/hllMS_dau.json","s3://mozilla-metrics/user/sguha/tmp/"])

In [ ]:
u2.write.save("s3://mozilla-metrics/tmp/bug1314436", format="csv")